# Different ways to make a query on mongoDB on ***C#***
The idea is create the same query using different techniques and try understanding the benefits of each one. 


## Setup 
___
***Reference***
* [mongo driver doc](https://docs.mongodb.com/drivers/csharp) 

### Dependencies

#### Mongo DB - DOCKER

In [1]:
#!pwsh
docker run -d -p 27017-27017:27017-27017 --name mongo-test mongo 

#### Nuget packages

In [1]:
#r "nuget:MongoDB.Driver"

Installed package MongoDB.Driver version 2.11.2

#### Client 

In [1]:

using MongoDB.Driver;

var client = new MongoClient("mongodb://localhost:27017");
var database = client.GetDatabase("firstDatabase");

### Creating Models

In [1]:
using MongoDB.Bson.Serialization.Attributes;

[BsonIgnoreExtraElements]
public class FirstCollection {
    public string FirstProperty { get; set; }
    public int SecondProperty { get; set; }
}
public class SecondCollection : FirstCollection {
    public string Name { get; set; }
}

### Seeding

In [1]:

var firstCollection = database.GetCollection<FirstCollection>("firstCollection");

var firstCollections = new List<FirstCollection>{
    new FirstCollection {FirstProperty = "First", SecondProperty = 1 },
    new FirstCollection {FirstProperty = "Second", SecondProperty = 2 },
    new FirstCollection {FirstProperty = "Third", SecondProperty = 3 },
    new FirstCollection {FirstProperty = "Fourth", SecondProperty = 4 }
};

firstCollection.InsertMany(firstCollections);

### Utils methods

In [1]:

    public void PrintResult<TResult>(List<TResult> results, Func<TResult,string> output){
        Console.WriteLine($"Result Count: {results.Count()}");
        results.ForEach(res => Console.WriteLine(output(res)));
    }


## Queries:

1. Simple query: The collections will filtered by property ***'FirstProperty'*** not equals ***'First'*** and the property ***'SecondProperty'*** in 1,2 and 4.
2. Aggragate query: Aggragate the  ***FirstCollection*** with ***SecondCollection*** by ***'FirstProperty'*** and filter ***FirstCollection*** by property ***'FirstProperty'*** not equals ***'First'***, selecting only ***SecondCollection's*** Name property.

### Using ***Filters*** Builder 
***Reference:***
* [Mongo DB Blog](https://www.mongodb.com/blog/post/quick-start-c-and-mongodb--read-operations)

In [1]:
var numbers = new List<int>{1,4,2};

var collectionInFilter = Builders<FirstCollection>.Filter.In(collect => collect.SecondProperty,numbers);
var collectionEqFilter = Builders<FirstCollection>.Filter.Eq(collect => collect.FirstProperty,"First");
var collectionNotInFilter = Builders<FirstCollection>.Filter.Not(collectionEqFilter);
var collectionAndFilter = Builders<FirstCollection>.Filter.And(collectionInFilter,collectionNotInFilter);

var findFilterResult = firstCollection.FindSync(collectionAndFilter).ToList();

PrintResult(findFilterResult, result => $"FirstProperty: {result.FirstProperty}, SecondProperty: {result.SecondProperty}");

### Using fluent ***Mongo driver*** Linq 
___
***Reference***
* [Mongo driver Doc LINQ](http://mongodb.github.io/mongo-csharp-driver/2.7/reference/driver/crud/linq/)

### Simple Query 
Use a simple filter with ***Linq*** as ***DSL*** not a good choice, because loses the great deal of this technique, the ***readability***.

In [1]:
var numbers = new List<int>{1,4,2};

var fluentResult = firstCollection.AsQueryable()
    .Where(collect => 
        !collect.FirstProperty.Equals("First")
        && numbers.Contains(collect.SecondProperty))
    .ToList();

PrintResult(fluentResult, result => $"FirstProperty: {result.FirstProperty}, SecondProperty: {result.SecondProperty}");


### Using DSL ***Mongo driver*** Linq 
***Reference***
* [Mongo driver Doc LINQ](http://mongodb.github.io/mongo-csharp-driver/2.7/reference/driver/crud/linq/)

In [1]:
var numbers = new List<int>{1,4,2};

var collectionQuerable = firstCollection.AsQueryable();

var dlsResult = from collect in collectionQuerable
                where !collect.FirstProperty.Equals("First")
                     && numbers.Contains(collect.SecondProperty)
                select collect;

PrintResult(dlsResult.ToList(), result => $"FirstProperty: {result.FirstProperty}, SecondProperty: {result.SecondProperty}");